# Data Importation

### Step Count Data

This dataset contains 90 days worth of step count data collected for each of the 84 participants depending on their assigned study group. Participants in "Blue" study group retained 90 days worth data taken from baseline whereas participants in "Red" study group retained 90 days worth of data taken starting from day 90. Data considered for each group were: Date, weight, BMI, and step count.

Step count data was imported from a folder containing 66 sub-folders -- one for each participant. Each subfolder contained 18 .csv files for each month of the trial.

In [2]:
#Uploads .xls files from each step count folder for each participant

import os

directory_list = []
for root, dirs, files in os.walk("/Users/ibergeland/Box Sync/Step-Omics/step count data files/", topdown = False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

import glob
import pandas as pd

step_count_activity = pd.DataFrame()
filenames = pd.DataFrame()
str = '/*.xls'
paths = [x + str for x in directory_list]
paths.sort()

From the Step Count spreadsheets, there are two tabs that we wish to select from the file for our resulting dataframe -- "Activities" & "Body".

#### Activities

In [3]:
for path in paths:
    filenames = glob.glob(path)
    for filename in filenames:
        df = pd.read_excel(filename, sheet_name='Activities')
        df['studyID']=path[60:64]
        step_count_activity = step_count_activity.append(df)

#Arranges by studyID and then Date in chronological order
step_count_activity['Date'] = pd.to_datetime(step_count_activity['Date'])
step_count_activity = step_count_activity.sort_values(['studyID','Date'])


In [8]:
cols = step_count_activity.columns.tolist()
cols = cols[-1:] + cols[:-1]
step_count_activity = step_count_activity[cols]
step_count_activity.head(1)

,Activity Calories,studyID,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active
0,407,2001,2014-11-17,"2,422","2,931",1.36,0.0,"1,364",31.0,40.0,5


#### Body

In [5]:
step_count_body=pd.DataFrame()
filenames=pd.DataFrame()
str='/*.xls'
paths=[x+str for x in directory_list]
paths.sort()
for path in paths:
    filenames = glob.glob(path)
    for filename in filenames:
        df=pd.read_excel(filename, sheet_name='Body')
        df['studyID']=path[60:64]
        step_count_body=step_count_body.append(df)
        
#Arranges by studyID and then Date in chronological order
step_count_body['Date'] = pd.to_datetime(step_count_body['Date'])
step_count_body = step_count_body.sort_values(['studyID','Date'])

In [9]:
#Moves studyID to first column
cols = step_count_body.columns.tolist()
cols = cols[-1:] + cols[:-1]
step_count_body = step_count_body[cols]
step_count_body.head(1)

,Fat,studyID,Date,Weight,BMI
0,0.0,2001,2014-11-17,253.4,36.24


#### Resulting Dataframe: Activity + Body

In [10]:
#This code merges the desired columns from step_count_activity and step_count_body and creates a single dataframe

#Selects 'studyID' and 'Steps' columns from step_count_activity tab
step_count_activityselect = step_count_activity[['studyID', 'Steps']].copy()

#Selects 'Date', 'Weight', and 'BMI' columns from step_count_body tab
step_count_bodyselect = step_count_body[['Date','Weight','BMI']].copy()

#Merges desired columns from step_count_activity and step_count_body
step_count = pd.concat([step_count_activityselect, step_count_bodyselect], axis=1, sort=False)

#Displays new step_count dataframe
step_count.head(1)

,studyID,Steps,Date,Weight,BMI
0,2001,"2,931",2014-11-17,253.4,36.24


### Anthropometrics Data

This dataset contains the studyID for each of the 84 participants as well as the check-in dates established baseline, 3 months, and 6 months. There is a "Red" or "Blue" study group associated with each participant that is used as a mapping for the other datasets.

In [11]:
#reads in original anthropometrics file
anthropometrics = pd.read_csv('F&T anthropometrics.csv')

#puts anthropometrics file in correct date format
anthropometrics['CorrectedDate'] =  pd.to_datetime(anthropometrics['CorrectedDate'], format='%d-%b-%y',errors='ignore')
anthropometrics['CorrectedRVDate'] = pd.to_datetime(anthropometrics['CorrectedRVDate'], format='%d-%b-%y',errors='ignore')

#drops unneeded columns from anthropometrics files
anthropometrics = anthropometrics.drop(columns = ['Date','RVDate','RVNotes'], axis = 1)

#creates new anthropometrics file, "anthro", with only the necessary columns included
anthro=anthropometrics[['studyID','CorrectedDate','StudyGroup']].copy()

#sorts anthro file by CorrectedDate and studyID
anthro['CorrectedDate'] = pd.to_datetime(anthro['CorrectedDate'])
anthro = anthro.sort_values(['studyID','CorrectedDate'])
anthro.head(1)

,studyID,CorrectedDate,StudyGroup
0,2001,2014-11-17,Blue


### Labs Data

This dataset contains demographic information such as: age, gender, baseline glucose and baseline A1C measurements taken for each of the 84 participants. Additionally there is the associated "Red" or "Blue" study group that we use to map between datasets.

In [12]:
#Imports original labs file
labs=pd.read_excel('F&T labs.xlsx')

#Corrects date format for Baseline, Month 3, Month 6
labs['BFastDate']=pd.to_datetime(labs['BFastDate'], format='%d-%b-%y',errors='ignore')
labs['MO3FastDate']=pd.to_datetime(labs['MO3FastDate'], format='%d-%b-%y',errors='ignore')
labs['MO6FastDate']=pd.to_datetime(labs['MO6FastDate'], format='%d-%b-%y',errors='ignore')

#Drops M06Validity column
labs=labs.drop(columns=['MO6Validity'],axis=1)

#Creates new Labs df with desired columns
Labs=labs[['studyID','StudyGroup','Age','Gender','Bglucose','BA1C','MO3Glucose','MO3A1C','MO6Glucose','MO6A1C']].copy()

#Display Labs
Labs.head(1)

,studyID,StudyGroup,Age,Gender,Bglucose,BA1C,MO3Glucose,MO3A1C,MO6Glucose,MO6A1C
0,2001,1,28,0,101,5.6,83,5.6,94,5.7


#### Separation - Red & Blue Labs 

Because participants were separated into 2 different groups, we needed to extract relevant data according to which group they were in. 

In [13]:
#Imports Python Numpy package
import numpy as np

#Specifies two new dataframes from Labs: Blue_labs & Red_labs according to their study group
Blue_labs = Labs[Labs.StudyGroup == 1]
Red_labs = Labs[Labs.StudyGroup == 2]

#Replaced 9999 values with NaN
Blue_labs = Blue_labs.replace(9999, np.nan)
Red_labs = Red_labs.replace(9999, np.nan)

#Drops irrelevant column labels
Blue_labs.drop(['MO6Glucose','MO6A1C'], axis=1, inplace=True)
Red_labs.drop(['Bglucose','BA1C'], axis=1, inplace=True)

#Creates dictionary to rename values to Blue and Red
mapping_studyGroup = {1: 'Blue', 2: 'Red'}
Blue_labs = Blue_labs.replace({'StudyGroup': mapping_studyGroup})
Red_labs = Red_labs.replace({'StudyGroup': mapping_studyGroup})

#Creates dictionary to rename Male and Female
mapping_gender = {0: 'Male', 1: 'Female'}
Blue_labs = Blue_labs.replace({'Gender': mapping_gender})
Red_labs = Red_labs.replace({'Gender': mapping_gender})

In [16]:
#Creates a copy of anthro file
anthro=anthropometrics[['studyID','CorrectedDate','StudyGroup']].copy()

#Creates a dictionary from anthro file to connect studyID to StudyGroup
mapping = dict(anthro[['studyID','StudyGroup']].values)

#Creates new column in step_count with StudyGroup label
step_count['StudyGroup'] = step_count.studyID.replace(mapping, inplace = True)

In [17]:
#Manually added study group
step_count_new = pd.read_csv('step_count_new.csv')
step_count_new.head(1)

,studyID,Steps,Date,Weight,BMI,StudyGroup
0,2001,"2,931",11/17/14,253.4,36.24,Blue


### Gene Expression 

This dataset contains the ensemble geneID gene expression for each of the 84 participants taken at baseline, 3 months, and 6 months. We normalized this dataset during feature engineering to contained normalized counts per million of gene expression data. We utilized the external HUGO Database to rename ensemble geneID's into HGNC gene ID's so that would could implement KEGG Pathways analysis of our gene expression data.

In [19]:
#Imports Original Gene Expression data (counts file)
counts = pd.read_csv('counts.csv', index_col = 0)

#Drops first four columns
gene_counts_df = counts.drop(labels = ['N_unmapped', 'N_multimapping', 'N_noFeature', 'N_ambiguous'], axis = 0)

#Display
gene_counts_df.head(1)

,2001-01,2001-02,2001-03,2002-01,2002-02,2002-03,2003-01,2003-02,2003-03,2004-01,...,2082-03,2083-01,2083-02,2083-03,2084-01,2084-02,2084-03,2085-01,2085-02,2085-03
ENSG00000223972.5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
